In [1]:
import pandas as pd
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors

!pip install geopandas==0.8.0
!pip install geojsonio==0.0.3

import geopandas as gpd


In [2]:
!conda install -c conda-forge folium=0.11.0 --yes
import folium

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.8.4

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [3]:
coviddata = "https://c19downloads.azureedge.net/downloads/msoa_data/MSOAs_latest.csv"
dfCovid = pd.read_csv(coviddata, index_col='msoa11_cd')
dfCovid.head()

,rgn19_cd,rgn19_nm,utla19_cd,utla19_nm,lad19_cd,lad19_nm,msoa11_hclnm,wk_05,wk_06,wk_07,...,wk_24,wk_25,wk_26,wk_27,wk_28,wk_29,wk_30,wk_31,wk_32,latest_7_days
msoa11_cd,,,,,,,,,,,,,,,,,,,,,
E02000001,E12000007,London,E09000001,City of London,E09000001,City of London,City of London,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E02000002,E12000007,London,E09000002,Barking and Dagenham,E09000002,Barking and Dagenham,Marks Gate,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN
E02000003,E12000007,London,E09000002,Barking and Dagenham,E09000002,Barking and Dagenham,Chadwell Heath East,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,4.0,4.0
E02000004,E12000007,London,E09000002,Barking and Dagenham,E09000002,Barking and Dagenham,Eastbrookend,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E02000005,E12000007,London,E09000002,Barking and Dagenham,E09000002,Barking and Dagenham,Becontree Heath,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
dfCovid = dfCovid[['msoa11_hclnm','lad19_nm','latest_7_days']]
dfCovid['latest_7_days'] = dfCovid['latest_7_days'].fillna(0)
dfCovid = dfCovid.reset_index()
dfCovid.rename(columns={'msoa11_cd':'MSOA01CD','msoa11_hclnm':'MSOA name','lad19_nm':'District','latest_7_days':'Infections (last 7 days)'}, inplace=True)
dfCovid.head()

,MSOA01CD,MSOA name,District,Infections (last 7 days)
0,E02000001,City of London,City of London,0.0
1,E02000002,Marks Gate,Barking and Dagenham,0.0
2,E02000003,Chadwell Heath East,Barking and Dagenham,4.0
3,E02000004,Eastbrookend,Barking and Dagenham,0.0
4,E02000005,Becontree Heath,Barking and Dagenham,0.0


In [24]:
pd.set_option('display.max_rows', None)
dfCovid.head()

,MSOA01CD,MSOA name,District,Infections (last 7 days)
0,E02000001,City of London,City of London,0.0
1,E02000002,Marks Gate,Barking and Dagenham,0.0
2,E02000003,Chadwell Heath East,Barking and Dagenham,4.0
3,E02000004,Eastbrookend,Barking and Dagenham,0.0
4,E02000005,Becontree Heath,Barking and Dagenham,0.0


*Noticed that one row is used to declare infections which are not attributed to a particular MSOA. This data is not relevant to my project as it cannot be displayed on a map of England. Further, it cannot be used to inform the lockdown of any establishment by geography. Code below removes this row and any others that might be lurking - in the end, just this one.*

In [6]:
dfCovid.drop(dfCovid[dfCovid['MSOA name'] == 'unallocated'].index, inplace = True)
dfCovid.shape

(6791, 4)

*The dataload was so heavy on memory that I needed to scale back my project to focus on one city area: Sheffield.*

In [53]:
pd.set_option('display.max_rows', None)
dfCovidSheffield = dfCovid.iloc[1572:1636]
dfCovidSheffield.head()

,MSOA01CD,MSOA name,District,Infections (last 7 days)
1572,E02001611,Stocksbridge,Sheffield,0.0
1573,E02001612,Deepcar & Bolsterstone,Sheffield,0.0
1574,E02001613,High Green & Burncross,Sheffield,0.0
1575,E02001614,Chapeltown,Sheffield,0.0
1576,E02001615,Grenoside & Ecclesfield North,Sheffield,0.0


In [27]:
dfCovidSheffield.shape

(64, 4)

Using an API with the Office of National Statistics, I can access the geojson file containing boundary data for each MSOA. Notice that the codes for each MSOA (below) match those in the Covid table (above).

In [9]:
uk_geo = gpd.read_file('https://opendata.arcgis.com/datasets/7923d7e44c394653b3ba794699d50854_0.geojson')


In [52]:
sheffield_geo = uk_geo[uk_geo['MSOA01NM'].str.contains('Sheffield')]
sheffield_geo.head()

,OBJECTID,MSOA01CD,MSOA01NM,MSOA01NMW,Shape__Area,Shape__Length,geometry
1610,1611,E02001611,Sheffield 001,Sheffield 001,5.717854e+06,15167.789019,"POLYGON ((-1.59042 53.47558, -1.59024 53.47529..."
1611,1612,E02001612,Sheffield 002,Sheffield 002,1.331386e+07,23325.723595,"POLYGON ((-1.59024 53.47529, -1.59042 53.47558..."
1612,1613,E02001613,Sheffield 003,Sheffield 003,4.273994e+06,13505.526379,"POLYGON ((-1.47423 53.46789, -1.47485 53.46779..."
1613,1614,E02001614,Sheffield 004,Sheffield 004,5.321935e+06,13660.654057,"POLYGON ((-1.48012 53.45705, -1.48011 53.45723..."
1614,1615,E02001615,Sheffield 005,Sheffield 005,1.109162e+07,24251.369538,"POLYGON ((-1.51492 53.46213, -1.51502 53.46233..."


In [51]:
dfmerged = sheffield_geo.merge(dfCovidSheffield,on='MSOA01CD',how='left')
dfSheffield = dfmerged.dropna()
dfSheffield.head()


,OBJECTID,MSOA01CD,MSOA01NM,MSOA01NMW,Shape__Area,Shape__Length,geometry,MSOA name,District,Infections (last 7 days)
0,1611,E02001611,Sheffield 001,Sheffield 001,5.717854e+06,15167.789019,"POLYGON ((-1.59042 53.47558, -1.59024 53.47529...",Stocksbridge,Sheffield,0.0
1,1612,E02001612,Sheffield 002,Sheffield 002,1.331386e+07,23325.723595,"POLYGON ((-1.59024 53.47529, -1.59042 53.47558...",Deepcar & Bolsterstone,Sheffield,0.0
2,1613,E02001613,Sheffield 003,Sheffield 003,4.273994e+06,13505.526379,"POLYGON ((-1.47423 53.46789, -1.47485 53.46779...",High Green & Burncross,Sheffield,0.0
3,1614,E02001614,Sheffield 004,Sheffield 004,5.321935e+06,13660.654057,"POLYGON ((-1.48012 53.45705, -1.48011 53.45723...",Chapeltown,Sheffield,0.0
4,1615,E02001615,Sheffield 005,Sheffield 005,1.109162e+07,24251.369538,"POLYGON ((-1.51492 53.46213, -1.51502 53.46233...",Grenoside & Ecclesfield North,Sheffield,0.0


In [50]:
#MSOA_name = dfCovidSheffield['MSOA name']
#sheffield_geo = sheffield_geo.join(MSOA_name)
#infections = dfCovidSheffield['Infections (last 7 days)']
#sheffield_geo = sheffield_geo.join(infections)
#sheffield_geo

In [49]:
#dfSheffieldmerged = pd.merge(dfCovidSheffield, sheffield_geo, on='MSOA01CD')
#dfSheffieldmerged
#pd.set_option('display.max_rows', None)
#dfSheffieldmerged.to_json(r'c:\\Users\epjac\Downloads\merged.json')

In [ ]:
#dfSheffieldmerged = pd.concat([dfCovidSheffield, sheffield_geo], axis=1).to_json()

In [48]:
import folium
from folium import plugins
uk_map = folium.Map(location=[53.3831, -1.4645], zoom_start=11)
choropleth = folium.Choropleth(
    name='Sheffield Covid Data',
    geo_data=dfSheffield,
    data= dfSheffield,
    columns=['MSOA01CD','Infections (last 7 days)'],
    key_on='feature.properties.MSOA01CD',
    fill_color='YlOrRd', 
    fill_opacity=0.4, 
    line_opacity=0.5,
    legend_name='SHEFFIELD - Covid infections in last 7 days',
    highlight=True
).add_to(uk_map)

choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['MSOA name','Infections (last 7 days)'])
)

uk_map
